In [ ]:
# Imports + load raw

import pandas as pd
import numpy as np

RAW_PATH = "../data/netflix_titles.csv"
df = pd.read_csv(RAW_PATH)
print(df.shape)
df.head(2)

(8807, 12)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."


In [5]:
# Helper (Missingness)

def missing_report(frame: pd.DataFrame, top=20):
    m = frame.isna().sum().sort_values(ascending=False)
    pct = (m / len(frame)).round(3)
    out = pd.DataFrame({"missing": m, "pct": pct})
    return out[out.missing > 0].head(top)

missing_report(df)

,missing,pct
director,2634,0.299
country,831,0.094
cast,825,0.094
date_added,10,0.001
month_added,10,0.001
rating,4,0.000
duration,3,0.000


In [ ]:
# Trim whitespace in all text columns

obj_cols = df.select_dtypes(include="object").columns
for c in obj_cols:
    df[c] = df[c].astype(str).str.strip().replace({"nan": np.nan})

In [10]:
# Parse date_added -> datetime + time features

df["date_added"] = pd.to_datetime(df["date_added"], errors="coerce")
df["year_added"] = df["date_added"].dt.year
df["month_added"] = df["date_added"].dt.month
df["year_added"] = df["year_added"].fillna(df["year_added"].median())

missing_report(df)

,missing,pct
director,2634,0.299
country,831,0.094
cast,825,0.094
date_added,10,0.001
month_added,10,0.001
rating,4,0.000
duration,3,0.000


In [11]:
# Ensure type and rating are tidy

df["type"] = df["type"].str.title() # 'Movie' / 'Tv Show' -> 'Movie'/'Tv Show'
df["type"] = df["type"].replace({"Tv Show": "TV Show"})

# Ratings: strip extra spaces, keep canonical labels as-is
df["rating"] = df["rating"].str.upper().str.replace(" ", "", regex=False)

# Ma[ known variants back to canonical (optional tweak list)
rating_map = {
    "TVY7": "TV-Y7", "TVPG": "TV-PG", "TVG": "TV-G", "TVMA": "TV-MA",
    "TV14": "TV-14", "PG13": "PG-13"
}
df["rating"] = df["rating"].replace(rating_map)
df["rating"] = df["rating"].replace({"NONE": np.nan, "NR": "NR", "UR": "UR"}) # Keep NR/UR if present
df["rating"].value_counts().head(10)

rating
TV-MA    3207
TV-14    2160
TV-PG     863
R         799
PG-13     490
TV-Y7     334
TV-Y      307
PG        287
TV-G      220
NR         80
Name: count, dtype: int64

In [ ]:
# Split duration into numeric fields
# Movies: minutes -> duration_min
# TV Shows: seasons -> seasons

# Initialize
df["duration_min"] = np.nan
df["seasons"] = np.nan

is_movie = df["type"] == "Movie"
is_show = df["type"] == "TV Show"

# Movies: "90 min"
df.loc[is_movie, "duration_min"] = (
    df.loc[is_movie, "duration"]
    .str.extract(r"(\d+)")
    .astype(float)
)

# TV Shows: "2 Seasons / "1 Season"
df.loc[is_show, "seasons"] = (
    df.loc[is_show, "duration"]
    .str.extract(r"(\d+)")
    .astype(float)
)

df[["type", "duration", "duration_min", "seasons"]].head(6)